In [103]:
import os
import re
import torch
import torch.nn.functional as F
from torch import nn
import pandas as pd
import matplotlib.pyplot as plt # for making figures
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from pprint import pprint

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [21]:
device

device(type='cpu')

In [197]:
folder_path = r'./Taylor-Swift-Songs'

# Loop through all files in the folder
songs = []

for file_name in os.listdir(folder_path):
    if file_name.endswith('.txt'):  # Check if the file is a .txt file
        file_path = os.path.join(folder_path, file_name)
        # Open and read the file
        
        with open(file_path, 'r', encoding='utf-8') as file:
            
            file_content = file.read()
            songs.append(file_content)


In [198]:
# Removing the first line in the song
# Replacing the '\n' with '<NEWLINE> token'

unique_words = set()
for i in range(len(songs)):
    lines = songs[i].splitlines()
    for j in range(len(lines)):
        # print(j,lines[j])
        lines[j] = re.sub(r'[^a-zA-Z0-9 .]', '', lines[j])
        lines[j] = lines[j].lower()
    songs[i] = " <newline> ".join(lines[1:])
    song_unique_words = set(word.lower() for word in songs[i].split())
    unique_words.update(song_unique_words)
    
    # print(len(song_unique_words))


In [199]:
len(unique_words)
for index,word in enumerate(unique_words):
    print(index,word)

0 dutch
1 opinion
2 gatlin
3 boaster.
4 chaps
5 closed
6 jeckles
7 draw
8 cascaded
9 gift
10 arm
11 shined
12 labyrinth
13 warn
14 sleepless
15 modest
16 attend
17 emperors
18 thank
19 mismatched
20 world
21 penthouse
22 dealing
23 depart
24 losing
25 slideshows
26 rustling
27 mike
28 hooligan
29 armor
30 asellum
31 neverhave
32 comrogue
33 whod
34 sanctimoniously
35 workman
36 cry
37 darling
38 childrens
39 defense
40 male
41 vi
42 usurer
43 conceit
44 graceless
45 ruld
46 laboratory
47 precede
48 won
49 columbia
50 knack
51 pragmatism
52 sow
53 54
54 dandies
55 magician
56 casually
57 andtie
58 rascals
59 wholl
60 critique
61 babys
62 layer
63 2003
64 herbwoman
65 picturing
66 hairpin
67 cheered
68 scattering
69 fusion
70 ga
71 carnationcolourd
72 those
73 industry
74 foolishly
75 wondred
76 lowerend
77 tyrants
78 suitors
79 ahead
80 brokenness
81 confessions
82 exceed
83 bastard
84 worship
85 list
86 grundles
87 sauces
88 gaze
89 devices
90 inferno
91 likeliest
92 clothing
93 toward

In [200]:

sorted_unique_words = sorted(unique_words)
for index,word in enumerate(unique_words):
    print(index,word)
Vocabulary = {word : index for index, word in enumerate(sorted_unique_words)}
iVocabulary = {index: word for index, word in enumerate(sorted_unique_words)}

len(Vocabulary)

0 dutch
1 opinion
2 gatlin
3 boaster.
4 chaps
5 closed
6 jeckles
7 draw
8 cascaded
9 gift
10 arm
11 shined
12 labyrinth
13 warn
14 sleepless
15 modest
16 attend
17 emperors
18 thank
19 mismatched
20 world
21 penthouse
22 dealing
23 depart
24 losing
25 slideshows
26 rustling
27 mike
28 hooligan
29 armor
30 asellum
31 neverhave
32 comrogue
33 whod
34 sanctimoniously
35 workman
36 cry
37 darling
38 childrens
39 defense
40 male
41 vi
42 usurer
43 conceit
44 graceless
45 ruld
46 laboratory
47 precede
48 won
49 columbia
50 knack
51 pragmatism
52 sow
53 54
54 dandies
55 magician
56 casually
57 andtie
58 rascals
59 wholl
60 critique
61 babys
62 layer
63 2003
64 herbwoman
65 picturing
66 hairpin
67 cheered
68 scattering
69 fusion
70 ga
71 carnationcolourd
72 those
73 industry
74 foolishly
75 wondred
76 lowerend
77 tyrants
78 suitors
79 ahead
80 brokenness
81 confessions
82 exceed
83 bastard
84 worship
85 list
86 grundles
87 sauces
88 gaze
89 devices
90 inferno
91 likeliest
92 clothing
93 toward

10746

In [201]:
Vocabulary

{'...here.': 0,
 '1': 1,
 '1000': 2,
 '10miles': 3,
 '11': 4,
 '112': 5,
 '13th': 6,
 '15': 7,
 '1500s': 8,
 '158': 9,
 '16': 10,
 '16th': 11,
 '17': 12,
 '18': 13,
 '1830s': 14,
 '1944': 15,
 '1950s': 16,
 '1958': 17,
 '1989': 18,
 '1989.': 19,
 '19th': 20,
 '2': 21,
 '20': 22,
 '20000': 23,
 '2003': 24,
 '2014': 25,
 '2017': 26,
 '2023': 27,
 '20year': 28,
 '230': 29,
 '230am': 30,
 '25': 31,
 '29th': 32,
 '2am': 33,
 '3': 34,
 '30s': 35,
 '31st': 36,
 '34': 37,
 '36th': 38,
 '4': 39,
 '43': 40,
 '45': 41,
 '4th': 42,
 '5': 43,
 '50000': 44,
 '51': 45,
 '54': 46,
 '555': 47,
 '6': 48,
 '601embed': 49,
 '60you': 50,
 '6th': 51,
 '7': 52,
 '75': 53,
 '805': 54,
 '80s': 55,
 '8bit': 56,
 '90s': 57,
 '98': 58,
 '<newline>': 59,
 'a': 60,
 'a.m': 61,
 'a.m.': 62,
 'aaron': 63,
 'ab': 64,
 'abandoned': 65,
 'abashd': 66,
 'abed': 67,
 'abide': 68,
 'abigail': 69,
 'ablaze': 70,
 'able': 71,
 'aboard': 72,
 'abomination': 73,
 'about': 74,
 'about.': 75,
 'above': 76,
 'abroad': 77,
 'absen

In [202]:
songs

['the drought was the very worst ohoh ohoh <newline> when the flowers that wed grown together died of thirst <newline> it was months and months of back and forth ohoh ohoh <newline> youre still all over me <newline> like a winestained dress i cant wear anymore <newline>  <newline> prechorus <newline> hung my head as i lost the war <newline> and the sky turned black like a perfect storm <newline> chorus <newline> rain came pouring down <newline> when i was drowning thats when i could finally breathe <newline> and by morning <newline> gone was any trace of you i think i am finally clean <newline> oh oh oh oh <newline>  <newline> verse 2 <newline> there was nothing left to do ohoh ohoh <newline> when the butterflies turned to <newline> dust that covered my whole room <newline> so i punched a hole in the roof ohoh ohoh <newline> let the flood carry away all my pictures of you <newline>  <newline> prechorus <newline> the water filled my lungs i screamed so loud <newline> but no one heard a 

In [ ]:
block_size = 200 # context length: how many characters do we take to predict the next one?
X, Y = [], []

for song in songs:
  
  # print(song)
  context = [False] * block_size
  song = song.split(' ')
  for word in song:
    if word == '':
      continue
    # print(word)
    ix = Vocabulary[word]
    X.append(context)
    Y.append(ix)
    print(' '.join('.' if not i else iVocabulary[i] for i in context), '--->', iVocabulary[ix])
    context = context[1:] + [ix] # crop and append
  
# Move data to GPU

X = torch.tensor(X).to(device)
Y = torch.tensor(Y).to(device)

tensor([[    0,     0,     0,  ...,     0,     0,     0],
        [    0,     0,     0,  ...,     0,     0,  9398],
        [    0,     0,     0,  ...,     0,  9398,  2835],
        ...,
        [ 6182,  9565,    59,  ...,    59,   462, 10673],
        [ 9565,    59,   462,  ...,   462, 10673,  7515],
        [   59,   462, 10673,  ..., 10673,  7515,  3583]])